In [18]:
from functools import reduce
from operator import add

import pandas as pd
import numpy as np

# Exercise 1
## Z: Inter-industry transactions of intermediate products (unit: €/year)

Intermediate products: goods and services as inputs in the production processes.

In [19]:
# Inter-industry transactions
Z_labels = pd.Index(["Agriculture", "Manufacturing", "Services"])
Z_values = [
    [0.6,2.6,0.5],
    [0.8, 30.6, 7.8],
    [0.9,12.1,23]
]

Z = pd.DataFrame(Z_values, index=Z_labels, columns=Z_labels)
Z

,Agriculture,Manufacturing,Services
Agriculture,0.6,2.6,0.5
Manufacturing,0.8,30.6,7.8
Services,0.9,12.1,23.0


## V: Value added (unit: €/year)

- Economic gains generated by producing goods and services.
- Main components: wages, taxes minus subsidies, capital profits, etc
- v: value added pet unit per year
- V: value added per year

In [20]:
# Value added (last row of the first table)
v_values = [3.30, 22.4, 52.5]

v = pd.Series(v_values, index=Z_labels)
v

Agriculture       3.3
Manufacturing    22.4
Services         52.5
dtype: float64

## Y: Final demand (unit: €/year)

In [21]:
# Final demand vector of products purchased by final consumers
y_values = [1.9, 28.5, 47.8]

y = pd.Series(y_values, index=Z_labels)
y

Agriculture       1.9
Manufacturing    28.5
Services         47.8
dtype: float64

## x: Gross output (unit: €/year)

Gross output of Sector j: intermediate products + finished products of Sector j

In [22]:
# x: Gross output (unit: €/year)
x_out = Z.sum(axis=1) + y
x_out

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

In [23]:
x_in = Z.sum() + v
x_in

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

## A: Technical coefficient (unit: €/€)

$$A = Z\hat{x}^{-1}$$
$$a_{i,j} = z_{i,j}/x_j$$
$$z_{i,j} = a_{i,j} x_j$$

$aij$ : direct requirements of sector i per € output of sector j  
$lij$ : total requirements of sector i per € final demand of sector j

In [24]:
# A (Unit: Million euro/Million euro)
A = Z.divide(x_out.T)  # Z = A.mul(x)
A.round(3)

,Agriculture,Manufacturing,Services
Agriculture,0.107,0.038,0.006
Manufacturing,0.143,0.452,0.093
Services,0.161,0.179,0.274


In [25]:
Id = pd.DataFrame(np.identity(len(Z_labels)), index=Z_labels, columns=Z_labels)
Id

,Agriculture,Manufacturing,Services
Agriculture,1.0,0.0,0.0
Manufacturing,0.0,1.0,0.0
Services,0.0,0.0,1.0


## L: Leontief inverse
### by inverse

$Z + y = x$  
$Ax + y = x$  
$y = (I-A)x$  
$(I-A)^{-1}(I-A)x=(I-A)^{-1}y$  

$x = (I-A)^{-1}y = Ly$

$lij$ : total requirements of sector i per € final demand of sector j

In [26]:
(Id - A)

,Agriculture,Manufacturing,Services
Agriculture,0.892857,-0.038405,-0.005967
Manufacturing,-0.142857,0.548006,-0.093079
Services,-0.160714,-0.178730,0.725537


In [27]:
L = pd.DataFrame(np.linalg.inv((Id - A).values), Z_labels, Z_labels)
L.round(2)

,Agriculture,Manufacturing,Services
Agriculture,1.14,0.09,0.02
Manufacturing,0.35,1.93,0.25
Services,0.34,0.49,1.44


### by series expansion

L covers supply chains of inifinte lengths. Power expansions stops at a certain depth

$$(I-A)^{-1} = I + A + A^2 + A^3 + ... + A^n + ... = \lim_{n \to +\infty} \sum_{k=0}^{n} A^k$$

The power series converges only under certain conditions:
- A ≥ 0 (coefficients matrix contains only non-negative terms)
- The system produces more output than it requires inputs N(A) < 1
- |I − A| > 0
- More generally, the [Hawkins-Simon conditions](https://www.wikiwand.com/en/Hawkins%E2%80%93Simon_condition)

the limit sum convergs if all of the eigenvalues of A have absolute value smaller than 1 (?)

In [28]:
# AA
A.dot(A).round(4)

,Agriculture,Manufacturing,Services
Agriculture,0.0179,0.0225,0.0059
Manufacturing,0.0948,0.2264,0.0685
Services,0.0869,0.1360,0.0929


In [29]:
# AAA
A.dot(A.dot(A)).round(4)

,Agriculture,Manufacturing,Services
Agriculture,0.0061,0.0119,0.0038
Manufacturing,0.0535,0.1182,0.0404
Services,0.0437,0.0814,0.0387


In [30]:
def leontief_power_series(A_mat, depth):
    Id_mat = pd.DataFrame(np.identity(len(Z_labels)), index=A.index, columns=A.columns)
    
    res = [None]*(depth+1)
    res[0] = Id_mat

    for i in range(1, depth+1):
        res[i] = res[i-1].dot(A_mat)
        print(f"{'A'*i}\n{res[i]}\n\n")

    return reduce(add, res)

In [31]:
leontief_power_series(A, 3)

A
               Agriculture  Manufacturing  Services
Agriculture       0.107143       0.038405  0.005967
Manufacturing     0.142857       0.451994  0.093079
Services          0.160714       0.178730  0.274463


AA
               Agriculture  Manufacturing  Services
Agriculture       0.017925       0.022540  0.005852
Manufacturing     0.094836       0.226421  0.068470
Services          0.086862       0.136012  0.092925


AAA
               Agriculture  Manufacturing  Services
Agriculture       0.006081       0.011922  0.003811
Manufacturing     0.053511       0.118221  0.040433
Services          0.043671       0.081421  0.038682




,Agriculture,Manufacturing,Services
Agriculture,1.131149,0.072867,0.015629
Manufacturing,0.291204,1.796636,0.201982
Services,0.291248,0.396162,1.406070


In [32]:
# Check that the total product output is equal to the one we have already calculated
L @ y

Agriculture       5.6
Manufacturing    67.7
Services         83.8
dtype: float64

## G: Ghosh quantity model

In [33]:
B = Z.divide(x_out.values[:, None])
B.round(3)

,Agriculture,Manufacturing,Services
Agriculture,0.107,0.464,0.089
Manufacturing,0.012,0.452,0.115
Services,0.011,0.144,0.274


In [34]:
G = pd.DataFrame(np.linalg.inv((Id - B).values), Z_labels, Z_labels)
G.round(2)

,Agriculture,Manufacturing,Services
Agriculture,1.14,1.04,0.31
Manufacturing,0.03,1.93,0.31
Services,0.02,0.40,1.44


In [37]:
# Check that the total product input is equal to the one we have already calculated x=G'v
x_g = G.T.dot(v)
# x_g = G.T @ v
np.isclose(x_g, x_out).all()

True

# Exercise 2
## Backward linkages

In [76]:
A

,Agriculture,Manufacturing,Services
Agriculture,0.107143,0.038405,0.005967
Manufacturing,0.142857,0.451994,0.093079
Services,0.160714,0.178730,0.274463


In [41]:
# Direct backward linkages
A.sum()

Agriculture      0.410714
Manufacturing    0.669129
Services         0.373508
dtype: float64

In [50]:
# Normalized Direct backward linkages
norm_dir_back_link = A.sum() / (A.sum().sum() / len(Z_labels))
norm_dir_back_link

Agriculture      0.847794
Manufacturing    1.381212
Services         0.770994
dtype: float64

In [51]:
# Total backward linkages
L.sum()

Agriculture      1.830853
Manufacturing    2.512672
Services         1.715695
dtype: float64

In [52]:
# Normalized Total backward linkages
norm_tot_back_link = L.sum() / (L.sum().sum() / len(Z_labels))
norm_tot_back_link

Agriculture      0.906479
Manufacturing    1.244057
Services         0.849463
dtype: float64

In [53]:
norm_backward_linkages = pd.concat([norm_dir_back_link, norm_tot_back_link], axis=1)
norm_backward_linkages.columns = ["direct_back_link", "total_back_link"]
norm_backward_linkages

,direct_back_link,total_back_link
Agriculture,0.847794,0.906479
Manufacturing,1.381212,1.244057
Services,0.770994,0.849463


## Forward linkages

In [54]:
# Direct forward linkages
B.sum(axis=1)

Agriculture      0.660714
Manufacturing    0.579025
Services         0.429594
dtype: float64

In [58]:
# Normalized Direct forward linkages
norm_dir_fwd_link = B.sum(axis=1) / (B.sum(axis=1).sum() / len(Z_labels))
norm_dir_fwd_link

Agriculture      1.187386
Manufacturing    1.040580
Services         0.772034
dtype: float64

In [59]:
# Total forward linkages
G.sum(axis=1)

Agriculture      2.487614
Manufacturing    2.270976
Services         1.867067
dtype: float64

In [60]:
# Normalized Total forward linkages
norm_tot_fwd_link = G.sum(axis=1) / (G.sum(axis=1).sum() / len(Z_labels))
norm_tot_fwd_link

Agriculture      1.126355
Manufacturing    1.028265
Services         0.845380
dtype: float64

In [61]:
norm_forward_linkages = pd.concat([norm_dir_fwd_link, norm_tot_fwd_link], axis=1)
norm_forward_linkages.columns = columns=["direct_fwd_link", "total_fwd_link"]
norm_forward_linkages

,direct_fwd_link,total_fwd_link
Agriculture,1.187386,1.126355
Manufacturing,1.040580,1.028265
Services,0.772034,0.845380


In [62]:
# Print normalized results for backward and forward linkages
pd.concat([norm_backward_linkages, norm_forward_linkages], axis=1).round(2)

,direct_back_link,total_back_link,direct_fwd_link,total_fwd_link
Agriculture,0.85,0.91,1.19,1.13
Manufacturing,1.38,1.24,1.04,1.03
Services,0.77,0.85,0.77,0.85
